In [2]:
import os
import pandas as pd
os.getcwd()

'/home/jupyter/nft/nft_capstone'

In [3]:
# import gcsfs, pickle

# fs = gcsfs.GCSFileSystem(project = 'w210-339016')
# fs.ls('nft_capstone_datafiles')

# with fs.open('nft_capstone_datafiles/Data_API.csv.gz', 'rb') as file:
#     df = pd.read_csv('/home/jupyter/nft/Data_API.csv.gz', dtype={'Smart_contract':str, 'ID_token':str, 'Transaction_hash':str,'Seller_username':str,'Buyer_username':str,
#                                                                  'Image_url_2':str,'Image_url_3':str,'Image_url_4':str,'Name':str,'Description':str,'Permanent_link':str})

df = pd.read_csv('https://storage.googleapis.com/nft_capstone_datafiles/Data_API.csv.gz',
                dtype={'Smart_contract':str, 'ID_token':str, 'Transaction_hash':str,'Seller_username':str,'Buyer_username':str,'Image_url_2':str,
                        'Image_url_3':str,'Image_url_4':str,'Name':str,'Description':str,'Permanent_link':str})

In [5]:
df['Image_url_1'].isna().value_counts()

False    6016229
True       54798
Name: Image_url_1, dtype: int64

In [4]:
df_images = df[['Unique_id_collection','Image_url_1']].copy()

In [5]:
df_images2 = df_images[df_images['Image_url_1'].isnull() == False]
df_images2.shape

(6016229, 2)

In [6]:
df_images5_new = df_images2.drop_duplicates(subset='Image_url_1', keep="first").drop(columns=['Unique_id_collection'])
print(len(df_images5_new))
df_images5_new.head(5)

1602953


,Image_url_1
0,https://card.godsunchained.com/?id=33&q=4
1,https://card.godsunchained.com/?id=79&q=4
3,https://card.godsunchained.com/?id=227&q=4
4,https://card.godsunchained.com/?id=62&q=4
5,https://card.godsunchained.com/?id=116&q=4


In [7]:
import requests
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable

In [8]:
model = models.alexnet(pretrained=True)
model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [9]:
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

In [10]:
def get_vector(image_url):
    # 1. Load the image with Pillow library
    r = requests.get(image_url, stream=True)
    img = Image.open(r.raw).convert('RGB')
    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(to_tensor(scaler(img))).unsqueeze(0)
    
    # 4. Define a function that will copy the output of a layer
    activation = {}
    def getActivation(name):
        # the hook signature
        def hook(model, input, output):
            activation[name] = output.detach().numpy()
        return hook
    # 5. Attach that function to our selected layer
    h = model.classifier[5].register_forward_hook(getActivation('ReLU'))
    # 6. Run the model on our transformed image
    model(t_img)
    # 7. Detach our copy function from the layer
    h.remove()
    # 8. Return the feature vector
    return activation['ReLU']

In [11]:
%%time
from numpy.lib.type_check import nan_to_num
from numpy.core.numeric import NaN

image_vectors = {}

# for val in range(len(df_images_test['Image_url_1'])):
#     image_vectors[df_images_test['ID_token'][val]] = get_vector(df_images_test['Image_url_1'][val])

# for val in range(10000,100000):
#     try:
#         image_vectors[df_images3['ID_token'].iloc[val]] = get_vector(df_images3['Image_url_1'].iloc[val])
#     except:
#         image_vectors[df_images3['ID_token'].iloc[val]] = NaN
        
for val in range(900000,1000000):
    try:
        image_vectors[df_images5_new['Image_url_1'].iloc[val]] = get_vector(df_images5_new['Image_url_1'].iloc[val])
    except:
        image_vectors[df_images5_new['Image_url_1'].iloc[val]] = NaN    

/opt/conda/lib/python3.7/site-packages/PIL/Image.py:976: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


CPU times: user 6h 33min 46s, sys: 1min 59s, total: 6h 35min 46s
Wall time: 5h 9min 53s


In [12]:
df_image_url_vectors = pd.DataFrame(list(image_vectors.items()), columns=['Image_url_1', 'img_vector'])

In [13]:
print(len(df_image_url_vectors))

100000


In [14]:
sum(pd.isnull(df_image_url_vectors['img_vector']))

3550

In [15]:
df_image_url_vectors.tail(5)

,Image_url_1,img_vector
99995,https://storage.opensea.io/gods-unchained-asse...,"[[3.8866713, 0.0, 0.0, 0.0, 0.14462747, 0.0, 0..."
99996,https://lh3.googleusercontent.com/eplPplnH_g9C...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.01..."
99997,https://storage.opensea.io/files/8b15e99f1a97a...,"[[0.2248167, 0.0, 0.0, 0.0, 0.4086939, 0.0, 0...."
99998,https://lh3.googleusercontent.com/tOBWoFKD482n...,"[[0.0, 0.0, 0.0, 0.0, 1.2215025, 0.0, 0.0, 0.0..."
99999,https://lh3.googleusercontent.com/q5PW-ouRdUKY...,"[[0.16260676, 0.0, 0.0, 0.0, 0.9754979, 0.0, 0..."


In [16]:
df_image_url_vectors.to_pickle('df_image_url_vectors900000-1000000.pkl')